# Power System Co-simulation Fundamentals

| Information | Details |
|----------|---------||
| Lead Author | Hantao Cui |
| Learning Objectives | • Understand transmission and distribution system characteristics<br>• Explain the need for separate power system simulators<br>• Implement basic T&D co-simulation using HELICS<br>• Configure data exchange between federates<br>• Validate co-simulation results<br>• Connect power system theory to practical simulation |
| Prerequisites | HELICS fundamentals, Python programming, basic power systems knowledge |
| Estimated Time | 180 minutes |
| Topics | Transmission systems, distribution systems, co-simulation architecture<br>ANDES, OpenDSS, HELICS integration, interface design |

## Introduction

Modern power systems are undergoing a fundamental transformation. The traditional one-way flow of electricity from large central generators through transmission networks to distribution systems is being replaced by a complex, bidirectional system. Distributed energy resources like rooftop solar, electric vehicles, and battery storage are creating new challenges for grid operators who must coordinate across multiple voltage levels and time scales.

This complexity has exposed a critical limitation in traditional power system analysis tools. Transmission system operators use specialized software optimized for high-voltage networks and system-wide stability analysis. Distribution utilities rely on different tools designed for radial networks, voltage regulation, and local reliability. These tools evolved independently because transmission and distribution systems historically operated with minimal interaction.

Today's grid requires a new approach. When a cloud passes over a large solar farm, the impact ripples through both distribution feeders and transmission lines. When electric vehicles charge simultaneously in a neighborhood, the effects can propagate upstream to transmission substations. Understanding these interactions requires co-simulation - the coordinated execution of multiple specialized simulators that exchange information to capture the full system behavior.

In this lesson, we bridge power system theory with practical co-simulation implementation. We'll start by understanding why transmission and distribution systems require different modeling approaches, then implement a basic co-simulation that demonstrates their interaction. By the end, you'll have both theoretical understanding and hands-on experience with transmission-distribution co-simulation using HELICS, ANDES, and OpenDSS.

## The Power System Hierarchy

Power systems operate across multiple scales, from continental interconnections down to individual households. This hierarchical structure isn't arbitrary - it reflects fundamental engineering trade-offs between efficiency, reliability, and cost. Understanding these different scales is essential for appreciating why co-simulation is necessary.

### Physical Characteristics

The most visible distinction between transmission and distribution systems is voltage level. Transmission systems operate at high voltages (69 kV to 800 kV) to minimize losses over long distances. Distribution systems step down these voltages (typically below 35 kV) to levels safe for customer delivery. But voltage is just the beginning of their differences.

Let's examine the key characteristics that distinguish these systems:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create a comparison table of transmission vs distribution characteristics
characteristics = {
    'Characteristic': ['Voltage Range', 'Geographic Scale', 'Network Topology', 
                      'Power Flow', 'Primary Function', 'Typical Customers'],
    'Transmission': ['69-800 kV', 'Regional/Interstate', 'Highly Meshed', 
                    'Bidirectional', 'Bulk Power Transfer', 'Utilities, Large Industrial'],
    'Distribution': ['120 V - 35 kV', 'Local/Neighborhood', 'Radial (mostly)', 
                    'Traditionally Unidirectional', 'Power Delivery', 'Residential, Commercial']
}

df = pd.DataFrame(characteristics)
print("Transmission vs Distribution System Characteristics")
print("=" * 80)
print(df.to_string(index=False))

### Operational Differences

Beyond physical characteristics, transmission and distribution systems operate on different principles and time scales. Transmission system operators focus on maintaining system-wide frequency stability and managing power transfers between regions. They deal with phenomena like generator synchronization, power system oscillations, and cascading failures that can affect millions of customers.

Distribution system operators concentrate on local voltage regulation and reliability. They manage equipment like capacitor banks, voltage regulators, and switches to maintain power quality for end users. The integration of distributed energy resources has added complexity, requiring distribution operators to handle reverse power flows and rapid voltage fluctuations.

These operational differences translate directly into modeling requirements. Transmission models emphasize dynamic behavior and system-wide interactions. Distribution models focus on detailed network representation and local power quality. This specialization is why we need different simulation tools for each domain.

In [ ]:
# Visualize the power system hierarchy
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

# Define system levels
levels = {
    'Generation': {'y': 5, 'voltage': '10-25 kV', 'color': '#2E86AB'},
    'Transmission': {'y': 4, 'voltage': '138-765 kV', 'color': '#A23B72'},
    'Subtransmission': {'y': 3, 'voltage': '34.5-138 kV', 'color': '#F18F01'},
    'Distribution Primary': {'y': 2, 'voltage': '4.16-34.5 kV', 'color': '#C73E1D'},
    'Distribution Secondary': {'y': 1, 'voltage': '120-480 V', 'color': '#6A994E'}
}

# Draw the hierarchy
for i, (level, props) in enumerate(levels.items()):
    # Draw level box
    rect = plt.Rectangle((1, props['y']-0.4), 8, 0.8, 
                        facecolor=props['color'], alpha=0.7, edgecolor='black')
    ax.add_patch(rect)
    
    # Add text
    ax.text(5, props['y'], level, ha='center', va='center', 
            fontsize=12, fontweight='bold', color='white')
    ax.text(10, props['y'], props['voltage'], ha='left', va='center', fontsize=10)
    
    # Draw connections
    if i < len(levels) - 1:
        ax.arrow(5, props['y']-0.4, 0, -0.2, head_width=0.2, head_length=0.1, 
                fc='black', ec='black')

# Add title and labels
ax.set_xlim(0, 12)
ax.set_ylim(0, 6)
ax.set_title('Power System Voltage Hierarchy', fontsize=16, fontweight='bold')
ax.text(0.5, 3, 'Power Flow', rotation=90, va='center', fontsize=12)
ax.axis('off')

# Add T-D interface indicator
ax.plot([0.5, 11.5], [3.5, 3.5], 'r--', linewidth=2)
ax.text(6, 3.7, 'T-D Interface', ha='center', color='red', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

## Why Separate Simulators?

The fundamental question students often ask is: why not model everything in one comprehensive simulator? The answer lies in computational efficiency, modeling fidelity, and practical software engineering considerations.

### Computational Complexity

Consider the scale difference between transmission and distribution systems. A regional transmission network might have 1,000 buses, while the distribution systems it serves could contain 100,000 or more nodes when modeled in detail. If we attempted to solve this entire system simultaneously, we would face several challenges.

First, the numerical properties of the combined system would be poor. Transmission lines have relatively low resistance-to-reactance ratios (R/X), while distribution systems have higher R/X ratios. Combining these in a single matrix leads to poor conditioning and convergence difficulties.

Second, the time scales of interest differ significantly. Transmission studies often require sub-second dynamics for stability analysis, while distribution planning studies might use hourly or 15-minute intervals. A unified simulator would need to accommodate the finest time scale across the entire system, leading to unnecessary computation.

### Modeling Philosophy

Different engineering communities have developed distinct modeling approaches optimized for their specific needs. Transmission engineers developed tools like PSS/E and PowerWorld that excel at dynamic security assessment and market operations. These tools use positive-sequence models that assume balanced three-phase operation - a reasonable approximation at transmission voltages.

Distribution engineers created tools like OpenDSS and CYME that handle unbalanced operation, detailed equipment models, and radial power flow algorithms. These tools can model individual customer loads and distributed energy resources with the detail needed for distribution planning.

Attempting to merge these modeling philosophies would compromise both. Transmission studies don't need three-phase unbalanced models for every distribution feeder, and distribution studies don't need generator swing equations for every transmission plant. Separate simulators allow each domain to use fit-for-purpose models.

In [ ]:
# Demonstrate computational scaling challenges
def estimate_computation_time(n_buses, is_transmission=True):
    """
    Estimate relative computation time for power flow based on system size.
    Uses O(n^1.4) scaling for sparse matrix operations.
    """
    base_time = 0.001  # seconds for small system
    
    if is_transmission:
        # Transmission systems are well-conditioned
        return base_time * (n_buses ** 1.4)
    else:
        # Distribution systems require more iterations due to high R/X
        return base_time * (n_buses ** 1.4) * 2.5

# Compare computation times
trans_buses = 1000
dist_buses = 50000
combined_buses = trans_buses + dist_buses

trans_time = estimate_computation_time(trans_buses, True)
dist_time = estimate_computation_time(dist_buses, False)
combined_time = estimate_computation_time(combined_buses, False)  # Poor conditioning

print("Estimated Computation Times for Power Flow:")
print(f"Transmission only ({trans_buses} buses): {trans_time:.3f} seconds")
print(f"Distribution only ({dist_buses} buses): {dist_time:.3f} seconds")
print(f"Separate simulation total: {trans_time + dist_time:.3f} seconds")
print(f"Combined simulation ({combined_buses} buses): {combined_time:.3f} seconds")
print(f"\nSpeedup from separation: {combined_time/(trans_time + dist_time):.1f}x")

## Manual Calculation Exercise

Before diving into automated co-simulation, let's build intuition by manually calculating power flow across a transmission-distribution interface. This exercise demonstrates the fundamental data exchange that co-simulation automates.

### Problem Setup

Consider a simple two-bus system where Bus 1 represents a transmission substation and Bus 2 represents an aggregated distribution load. The transmission bus maintains a voltage of 1.05 per unit at 0 degrees. The distribution system draws 50 MW and 10 MVAr. A line with impedance Z = 0.01 + j0.03 per unit connects them.

In [ ]:
# Manual power flow calculation
import cmath

# Given parameters
V1_mag = 1.05  # Transmission bus voltage magnitude (pu)
V1_angle = 0.0  # Transmission bus voltage angle (radians)
P_load = 50.0  # Distribution load active power (MW)
Q_load = 10.0  # Distribution load reactive power (MVAr)
Z_line = 0.01 + 0.03j  # Line impedance (pu)
S_base = 100.0  # Base power (MVA)

# Convert to per unit
P_load_pu = P_load / S_base
Q_load_pu = Q_load / S_base
S_load_pu = P_load_pu - 1j * Q_load_pu  # Note: load convention (consuming power)

# Step 1: Calculate voltage at transmission bus
V1 = V1_mag * cmath.exp(1j * V1_angle)
print(f"Step 1: Transmission bus voltage")
print(f"V1 = {V1_mag:.4f} ∠ {math.degrees(V1_angle):.1f}° pu")
print(f"V1 = {V1.real:.4f} + j{V1.imag:.4f} pu\n")

# Step 2: Initial estimate of current (assuming V2 ≈ V1)
I_line = S_load_pu.conjugate() / V1.conjugate()
print(f"Step 2: Initial current estimate")
print(f"I = S*/V1* = {abs(I_line):.4f} ∠ {math.degrees(cmath.phase(I_line)):.1f}° pu")
print(f"I = {I_line.real:.4f} + j{I_line.imag:.4f} pu\n")

# Step 3: Calculate voltage drop
V_drop = I_line * Z_line
print(f"Step 3: Voltage drop across line")
print(f"ΔV = I × Z = {abs(V_drop):.4f} ∠ {math.degrees(cmath.phase(V_drop)):.1f}° pu")
print(f"ΔV = {V_drop.real:.4f} + j{V_drop.imag:.4f} pu\n")

# Step 4: Calculate distribution bus voltage
V2 = V1 - V_drop
V2_mag = abs(V2)
V2_angle = cmath.phase(V2)
print(f"Step 4: Distribution bus voltage")
print(f"V2 = V1 - ΔV = {V2_mag:.4f} ∠ {math.degrees(V2_angle):.1f}° pu")
print(f"V2 = {V2.real:.4f} + j{V2.imag:.4f} pu\n")

# Step 5: Verify power balance
S_line_send = V1 * I_line.conjugate()
S_line_recv = V2 * I_line.conjugate()
S_loss = S_line_send - S_line_recv

print(f"Step 5: Power balance verification")
print(f"Power sent from transmission: {S_line_send.real*S_base:.2f} MW, {-S_line_send.imag*S_base:.2f} MVAr")
print(f"Power received at distribution: {S_line_recv.real*S_base:.2f} MW, {-S_line_recv.imag*S_base:.2f} MVAr")
print(f"Line losses: {S_loss.real*S_base:.2f} MW, {-S_loss.imag*S_base:.2f} MVAr")

### Key Insights from Manual Calculation

This simple calculation reveals several important concepts that carry over to full co-simulation:

1. **Data Exchange**: The transmission system provides voltage (magnitude and angle), while the distribution system provides power consumption. This defines the interface variables for co-simulation.

2. **Coupling**: Changes in distribution load directly affect transmission bus voltage through the line impedance. Similarly, transmission voltage variations impact distribution system operation.

3. **Iteration**: In practice, we would iterate this calculation because the distribution load might depend on voltage. Co-simulation handles this iteration automatically.

4. **Scaling**: Real systems have thousands of buses, but the fundamental principle remains: exchange voltage and power information at interface buses.

## Co-simulation Architecture Design

Now that we understand the manual process, let's design a co-simulation architecture that automates this data exchange. We'll start with a simplified system that demonstrates the key concepts without overwhelming complexity.

### System Configuration

For this educational example, we'll use:
- **Transmission**: A 3-bus system with one generator, one load, and one T-D interface bus
- **Distribution**: A 5-bus radial feeder with residential loads
- **Interface**: Bus 3 in transmission connects to Bus 1 in distribution

This simplified configuration allows us to focus on the co-simulation mechanics while still capturing realistic behavior.

In [ ]:
# Visualize the co-simulation architecture
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Transmission system diagram
ax1.set_title('Transmission System (ANDES)', fontsize=14, fontweight='bold')
ax1.set_xlim(-1, 5)
ax1.set_ylim(-1, 4)

# Transmission buses
trans_buses = [(0, 2), (2, 2), (4, 2)]  # Bus 1, 2, 3
for i, (x, y) in enumerate(trans_buses):
    color = '#2E86AB' if i < 2 else '#FF6B6B'  # Red for interface bus
    circle = plt.Circle((x, y), 0.3, color=color, alpha=0.7)
    ax1.add_patch(circle)
    ax1.text(x, y, f'Bus {i+1}', ha='center', va='center', fontweight='bold')

# Transmission lines
ax1.plot([0, 2], [2, 2], 'k-', linewidth=2)
ax1.plot([2, 4], [2, 2], 'k-', linewidth=2)

# Generator and load symbols
ax1.text(0, 3, 'G', ha='center', va='center', fontsize=16, 
         bbox=dict(boxstyle="circle", facecolor='yellow'))
ax1.text(2, 1, 'L', ha='center', va='center', fontsize=16,
         bbox=dict(boxstyle="square", facecolor='lightblue'))

# Labels
ax1.text(1, 2.3, '138 kV', ha='center', fontsize=10)
ax1.text(3, 2.3, '138 kV', ha='center', fontsize=10)
ax1.text(4, 0.5, 'To Distribution\n(Interface)', ha='center', color='red')
ax1.axis('off')

# Distribution system diagram
ax2.set_title('Distribution System (OpenDSS)', fontsize=14, fontweight='bold')
ax2.set_xlim(-1, 5)
ax2.set_ylim(-1, 4)

# Distribution buses
dist_buses = [(0, 2), (1.5, 2), (3, 2), (1.5, 3.5), (1.5, 0.5)]
for i, (x, y) in enumerate(dist_buses):
    color = '#FF6B6B' if i == 0 else '#6A994E'  # Red for interface
    circle = plt.Circle((x, y), 0.25, color=color, alpha=0.7)
    ax2.add_patch(circle)
    ax2.text(x, y, f'{i+1}', ha='center', va='center', fontweight='bold', fontsize=10)

# Distribution lines
ax2.plot([0, 1.5], [2, 2], 'k-', linewidth=1.5)
ax2.plot([1.5, 3], [2, 2], 'k-', linewidth=1.5)
ax2.plot([1.5, 1.5], [2, 3.5], 'k-', linewidth=1.5)
ax2.plot([1.5, 1.5], [2, 0.5], 'k-', linewidth=1.5)

# Load symbols
for (x, y) in [(3, 2), (1.5, 3.5), (1.5, 0.5)]:
    ax2.text(x+0.4, y, 'L', ha='center', va='center', fontsize=12,
             bbox=dict(boxstyle="square", facecolor='lightgreen'))

# Voltage levels
ax2.text(0.75, 2.3, '13.8 kV', ha='center', fontsize=9)
ax2.text(0, 0.5, 'From Transmission\n(Interface)', ha='center', color='red')
ax2.axis('off')

# Add data flow indicators
fig.text(0.5, 0.9, 'HELICS Data Exchange', ha='center', fontsize=12, fontweight='bold')
fig.text(0.5, 0.85, 'Voltage (V, θ) →', ha='center', fontsize=10, color='blue')
fig.text(0.5, 0.80, '← Power (P, Q)', ha='center', fontsize=10, color='green')

plt.tight_layout()
plt.show()

### Interface Definition

The interface between transmission and distribution systems is where co-simulation magic happens. At this boundary, we need to:

1. **From Transmission to Distribution**: Provide voltage magnitude and angle at the high-voltage side of the substation transformer
2. **From Distribution to Transmission**: Provide the total active and reactive power drawn by the distribution system

This creates a closed loop where transmission voltage affects distribution power consumption, which in turn affects transmission voltage. HELICS manages the timing and data exchange to ensure both simulators stay synchronized.

## HELICS Federation Setup

With our system architecture defined, let's implement the HELICS federation. We'll create two federates - one for transmission (ANDES) and one for distribution (OpenDSS) - that exchange data at each time step.

### Environment Verification

In [ ]:
# Verify required packages are available
import sys
import importlib

required_packages = {
    'helics': 'HELICS co-simulation framework',
    'andes': 'ANDES transmission system simulator',
    'opendssdirect': 'OpenDSS distribution system simulator'
}

missing_packages = []
for package, description in required_packages.items():
    try:
        importlib.import_module(package)
        print(f"✓ {package}: {description}")
    except ImportError:
        print(f"✗ {package}: Not installed - {description}")
        missing_packages.append(package)

if missing_packages:
    print("\nPlease install missing packages before proceeding.")
    print("Refer to Module 06 Lesson 1 for installation instructions.")
else:
    print("\nAll required packages are available. Ready to proceed!")
    
    # Import packages for use
    import helics as h
    import andes
    import opendssdirect as dss

### Creating System Models

Before implementing the co-simulation, we need to create our transmission and distribution system models. For educational purposes, we'll define these programmatically rather than loading from files.

In [ ]:
def create_simple_transmission_system():
    """
    Create a simple 3-bus transmission system for educational demonstration.
    Bus 1: Generator (slack bus)
    Bus 2: Load
    Bus 3: T-D interface (load will be replaced by distribution equivalent)
    """
    # This is a simplified example - in practice, you would load from a case file
    system_data = {
        'Bus': [
            {'idx': 1, 'name': 'Bus1', 'Vn': 138.0, 'voltage': 1.05, 'angle': 0.0, 'bus_type': 'slack'},
            {'idx': 2, 'name': 'Bus2', 'Vn': 138.0, 'voltage': 1.0, 'angle': 0.0, 'bus_type': 'PQ'},
            {'idx': 3, 'name': 'Bus3', 'Vn': 138.0, 'voltage': 1.0, 'angle': 0.0, 'bus_type': 'PQ'}
        ],
        'Line': [
            {'idx': 1, 'bus1': 1, 'bus2': 2, 'r': 0.01, 'x': 0.05, 'b': 0.02},
            {'idx': 2, 'bus1': 2, 'bus2': 3, 'r': 0.02, 'x': 0.08, 'b': 0.03}
        ],
        'PQ': [
            {'idx': 1, 'bus': 2, 'p': 0.3, 'q': 0.1},  # 30 MW, 10 MVAr load at Bus 2
            {'idx': 2, 'bus': 3, 'p': 0.5, 'q': 0.15}  # Initial load at interface (will be updated)
        ],
        'Generator': [
            {'idx': 1, 'bus': 1, 'p': 1.0, 'v': 1.05}  # Generator at Bus 1
        ]
    }
    
    return system_data

def create_simple_distribution_system():
    """
    Create a simple 5-bus radial distribution system.
    This will be implemented in OpenDSS format.
    """
    dss_commands = [
        "Clear",
        "New Circuit.Simple5Bus basekv=13.8 bus1=1",
        
        # Define lines
        "New Line.Line1 bus1=1 bus2=2 length=1 units=km r1=0.1 x1=0.3",
        "New Line.Line2 bus1=2 bus2=3 length=0.5 units=km r1=0.1 x1=0.3",
        "New Line.Line3 bus1=2 bus2=4 length=0.8 units=km r1=0.1 x1=0.3",
        "New Line.Line4 bus1=2 bus2=5 length=0.6 units=km r1=0.1 x1=0.3",
        
        # Define loads
        "New Load.Load3 bus1=3 kV=13.8 kW=2000 kvar=500",
        "New Load.Load4 bus1=4 kV=13.8 kW=1500 kvar=300",
        "New Load.Load5 bus1=5 kV=13.8 kW=1000 kvar=200",
        
        # Set solution mode
        "Set Voltagebases=[13.8]",
        "Calcvoltagebases",
        "Set Mode=Snap"
    ]
    
    return dss_commands

print("System creation functions defined.")
print("These functions create simplified models suitable for educational demonstration.")

### Transmission Federate Implementation

The transmission federate manages the high-voltage network simulation using ANDES. It receives power consumption data from the distribution system and provides voltage information in return.

In [ ]:
def transmission_federate(stop_time=10.0):
    """
    Transmission system federate using ANDES.
    Publishes: Voltage magnitude and angle at interface bus
    Subscribes: Total power from distribution system
    """
    import andes
    import helics as h
    import numpy as np
    
    print("[Transmission] Initializing federate...")
    
    # Create federate info object
    fedinfo = h.helicsCreateFederateInfo()
    h.helicsFederateInfoSetCoreName(fedinfo, "TransmissionFederate")
    h.helicsFederateInfoSetCoreTypeFromString(fedinfo, "zmq")
    h.helicsFederateInfoSetCoreInitString(fedinfo, "--federates=1")
    h.helicsFederateInfoSetTimeProperty(fedinfo, h.helics_property_time_delta, 1.0)
    
    # Create value federate
    fed = h.helicsCreateValueFederate("TransmissionFederate", fedinfo)
    print("[Transmission] Federate created")
    
    # Register publications - voltage at interface bus
    pub_voltage = h.helicsFederateRegisterGlobalPublication(
        fed, "transmission/voltage_magnitude", h.helics_data_type_double, "pu"
    )
    pub_angle = h.helicsFederateRegisterGlobalPublication(
        fed, "transmission/voltage_angle", h.helics_data_type_double, "rad"
    )
    
    # Register subscriptions - power from distribution
    sub_p = h.helicsFederateRegisterSubscription(fed, "distribution/active_power", "MW")
    sub_q = h.helicsFederateRegisterSubscription(fed, "distribution/reactive_power", "MVAr")
    
    print("[Transmission] Publications and subscriptions registered")
    
    # Initialize transmission system (simplified for demonstration)
    # In practice, you would load an ANDES case file here
    interface_bus_idx = 2  # Bus 3 (0-indexed)
    
    # Enter execution mode
    h.helicsFederateEnterExecutingMode(fed)
    print("[Transmission] Entered execution mode")
    
    # Simulation data storage
    time_data = []
    voltage_data = []
    power_data = []
    
    # Main simulation loop
    current_time = 0.0
    while current_time < stop_time:
        # Request time advancement
        current_time = h.helicsFederateRequestTime(fed, current_time + 1.0)
        
        # Get power from distribution system
        if h.helicsInputIsUpdated(sub_p) and h.helicsInputIsUpdated(sub_q):
            p_dist = h.helicsInputGetDouble(sub_p)
            q_dist = h.helicsInputGetDouble(sub_q)
        else:
            # Use default values for first iteration
            p_dist = 50.0  # MW
            q_dist = 15.0  # MVAr
        
        print(f"[Transmission] Time {current_time:.0f}s: Received P={p_dist:.1f} MW, Q={q_dist:.1f} MVAr")
        
        # Update load at interface bus (simplified calculation)
        # In practice, this would update the ANDES system and run power flow
        
        # For demonstration, calculate voltage based on simple drop
        # V = V0 - (P*R + Q*X)/V0 approximation
        v0 = 1.05  # Source voltage
        r_total = 0.03  # Total resistance to interface
        x_total = 0.13  # Total reactance to interface
        
        v_drop_real = (p_dist/100 * r_total + q_dist/100 * x_total) / v0
        v_drop_imag = (p_dist/100 * x_total - q_dist/100 * r_total) / v0
        
        v_magnitude = v0 - v_drop_real
        v_angle = -v_drop_imag  # Small angle approximation
        
        # Publish voltage to distribution system
        h.helicsPublicationPublishDouble(pub_voltage, v_magnitude)
        h.helicsPublicationPublishDouble(pub_angle, v_angle)
        
        print(f"[Transmission] Published V={v_magnitude:.4f} pu, θ={math.degrees(v_angle):.2f}°")
        
        # Store data
        time_data.append(current_time)
        voltage_data.append((v_magnitude, v_angle))
        power_data.append((p_dist, q_dist))
    
    # Finalize
    h.helicsFederateFree(fed)
    print("[Transmission] Federate finalized")
    
    return time_data, voltage_data, power_data

### Distribution Federate Implementation

The distribution federate manages the lower-voltage network using OpenDSS. It receives voltage conditions from the transmission system and calculates the total power demand.

In [ ]:
def distribution_federate(stop_time=10.0):
    """
    Distribution system federate using OpenDSS.
    Publishes: Total active and reactive power
    Subscribes: Voltage magnitude and angle from transmission
    """
    import opendssdirect as dss
    import helics as h
    import numpy as np
    
    print("[Distribution] Initializing federate...")
    
    # Create federate
    fedinfo = h.helicsCreateFederateInfo()
    h.helicsFederateInfoSetCoreName(fedinfo, "DistributionFederate")
    h.helicsFederateInfoSetCoreTypeFromString(fedinfo, "zmq")
    h.helicsFederateInfoSetCoreInitString(fedinfo, "--federates=1")
    h.helicsFederateInfoSetTimeProperty(fedinfo, h.helics_property_time_delta, 1.0)
    
    fed = h.helicsCreateValueFederate("DistributionFederate", fedinfo)
    print("[Distribution] Federate created")
    
    # Register publications - total power
    pub_p = h.helicsFederateRegisterGlobalPublication(
        fed, "distribution/active_power", h.helics_data_type_double, "MW"
    )
    pub_q = h.helicsFederateRegisterGlobalPublication(
        fed, "distribution/reactive_power", h.helics_data_type_double, "MVAr"
    )
    
    # Register subscriptions - voltage from transmission
    sub_voltage = h.helicsFederateRegisterSubscription(fed, "transmission/voltage_magnitude", "pu")
    sub_angle = h.helicsFederateRegisterSubscription(fed, "transmission/voltage_angle", "rad")
    
    print("[Distribution] Publications and subscriptions registered")
    
    # Initialize distribution system
    dss_commands = create_simple_distribution_system()
    for cmd in dss_commands:
        dss.run_command(cmd)
    
    print("[Distribution] System initialized")
    
    # Enter execution mode
    h.helicsFederateEnterExecutingMode(fed)
    print("[Distribution] Entered execution mode")
    
    # Get base load values
    dss.run_command("Solve")
    base_power = dss.Circuit.TotalPower()
    base_p = -base_power[0] / 1000  # Convert to MW
    base_q = -base_power[1] / 1000  # Convert to MVAr
    
    # Simulation data storage
    time_data = []
    voltage_data = []
    power_data = []
    
    # Main simulation loop
    current_time = 0.0
    while current_time < stop_time:
        # Request time advancement
        current_time = h.helicsFederateRequestTime(fed, current_time + 1.0)
        
        # Get voltage from transmission system
        if h.helicsInputIsUpdated(sub_voltage) and h.helicsInputIsUpdated(sub_angle):
            v_magnitude = h.helicsInputGetDouble(sub_voltage)
            v_angle = h.helicsInputGetDouble(sub_angle)
        else:
            # Use nominal values for first iteration
            v_magnitude = 1.0
            v_angle = 0.0
        
        print(f"[Distribution] Time {current_time:.0f}s: Received V={v_magnitude:.4f} pu, θ={math.degrees(v_angle):.2f}°")
        
        # Update source voltage in OpenDSS
        # Note: This is simplified - actual implementation would handle transformer, etc.
        dss.Vsources.PU(v_magnitude)
        dss.Vsources.AngleDeg(math.degrees(v_angle))
        
        # Add time-varying load (sinusoidal variation for demonstration)
        load_factor = 1.0 + 0.2 * math.sin(2 * math.pi * current_time / 10)
        
        # Update all loads
        dss.Loads.First()
        while True:
            original_kw = dss.Loads.kW()
            dss.Loads.kW(original_kw * load_factor)
            if not dss.Loads.Next():
                break
        
        # Solve distribution power flow
        dss.run_command("Solve")
        
        # Get total power
        total_power = dss.Circuit.TotalPower()
        p_total = -total_power[0] / 1000  # Convert to MW
        q_total = -total_power[1] / 1000  # Convert to MVAr
        
        # Publish power to transmission system
        h.helicsPublicationPublishDouble(pub_p, p_total)
        h.helicsPublicationPublishDouble(pub_q, q_total)
        
        print(f"[Distribution] Published P={p_total:.1f} MW, Q={q_total:.1f} MVAr")
        
        # Store data
        time_data.append(current_time)
        voltage_data.append((v_magnitude, v_angle))
        power_data.append((p_total, q_total))
    
    # Finalize
    h.helicsFederateFree(fed)
    print("[Distribution] Federate finalized")
    
    return time_data, voltage_data, power_data

### Running the Co-simulation

Now we'll execute the complete co-simulation. In a real implementation, the federates would run in separate processes or even separate machines. For this educational example, we'll use threading to run them concurrently in the same notebook.

In [ ]:
import threading
import time
import math
from queue import Queue

def run_basic_cosimulation(simulation_time=10.0):
    """
    Run the transmission-distribution co-simulation.
    """
    print("=" * 60)
    print("Starting Basic T&D Co-simulation")
    print("=" * 60)
    
    # Create HELICS broker
    print("\nCreating HELICS broker...")
    broker = h.helicsCreateBroker("zmq", "", "-f 2 --name=cosim_broker")
    
    if h.helicsBrokerIsConnected(broker):
        print("✓ Broker created and connected")
    else:
        print("✗ Broker connection failed")
        return None
    
    # Create queues to collect results from threads
    trans_queue = Queue()
    dist_queue = Queue()
    
    # Define wrapper functions that store results
    def run_transmission():
        try:
            results = transmission_federate(simulation_time)
            trans_queue.put(results)
        except Exception as e:
            print(f"[Transmission] Error: {e}")
            trans_queue.put(None)
    
    def run_distribution():
        try:
            results = distribution_federate(simulation_time)
            dist_queue.put(results)
        except Exception as e:
            print(f"[Distribution] Error: {e}")
            dist_queue.put(None)
    
    # Start federate threads
    print("\nStarting federates...")
    trans_thread = threading.Thread(target=run_transmission)
    dist_thread = threading.Thread(target=run_distribution)
    
    trans_thread.start()
    time.sleep(0.5)  # Small delay to ensure proper initialization order
    dist_thread.start()
    
    # Wait for completion
    print("\nCo-simulation running...")
    trans_thread.join(timeout=30)
    dist_thread.join(timeout=30)
    
    # Destroy broker
    h.helicsBrokerDestroy(broker)
    print("\n✓ Co-simulation completed")
    
    # Collect results
    trans_results = trans_queue.get() if not trans_queue.empty() else None
    dist_results = dist_queue.get() if not dist_queue.empty() else None
    
    return trans_results, dist_results

# Run the co-simulation
print("Note: This is a simplified demonstration. Real co-simulations would use:")
print("- Actual ANDES system models with full dynamics")
print("- Complete OpenDSS distribution networks")
print("- Proper scaling and unit conversions")
print("\nFor this educational example, we'll demonstrate the data flow concept.")
print()

# Store results for analysis
cosim_results = None

# Placeholder for actual execution
# In practice, you would uncomment the following line after ensuring all dependencies are installed:
# cosim_results = run_basic_cosimulation(10.0)

## Results Analysis and Visualization

After running the co-simulation, we need to analyze the results to understand the system behavior and validate our implementation. Let's create visualizations that show the interaction between transmission and distribution systems.

In [ ]:
# Demonstrate result analysis with simulated data
# In practice, this would use actual co-simulation results

def analyze_cosimulation_results(trans_results=None, dist_results=None):
    """
    Analyze and visualize co-simulation results.
    """
    
    # For demonstration, create example data showing typical behavior
    if trans_results is None or dist_results is None:
        print("Creating example data for visualization demonstration...")
        time_points = np.arange(0, 11, 1)
        
        # Simulated load variation (sinusoidal)
        load_factor = 1.0 + 0.2 * np.sin(2 * np.pi * time_points / 10)
        base_p = 50.0  # MW
        base_q = 15.0  # MVAr
        
        # Distribution power (varies with load factor)
        p_dist = base_p * load_factor
        q_dist = base_q * load_factor
        
        # Transmission voltage (responds to load)
        v_base = 1.05
        v_drop = 0.002 * (p_dist - base_p) / base_p  # 0.2% drop per 10% load increase
        v_trans = v_base - v_drop
        angle_trans = -0.05 * (p_dist - base_p) / base_p  # Angle change with load
        
        trans_results = (time_points, list(zip(v_trans, angle_trans)), list(zip(p_dist, q_dist)))
        dist_results = (time_points, list(zip(v_trans, angle_trans)), list(zip(p_dist, q_dist)))
    
    # Extract data
    time_data = trans_results[0]
    voltage_data = np.array(trans_results[1])
    power_data = np.array(trans_results[2])
    
    # Create comprehensive visualization
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))
    
    # Plot 1: Voltage magnitude over time
    ax1.plot(time_data, voltage_data[:, 0], 'b-', linewidth=2, marker='o', markersize=6)
    ax1.set_xlabel('Time (s)', fontsize=12)
    ax1.set_ylabel('Voltage Magnitude (pu)', fontsize=12)
    ax1.set_title('Interface Bus Voltage Magnitude', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim([1.045, 1.055])
    
    # Plot 2: Power demand over time
    ax2.plot(time_data, power_data[:, 0], 'g-', linewidth=2, marker='s', markersize=6, label='Active (P)')
    ax2.plot(time_data, power_data[:, 1], 'r--', linewidth=2, marker='^', markersize=6, label='Reactive (Q)')
    ax2.set_xlabel('Time (s)', fontsize=12)
    ax2.set_ylabel('Power (MW/MVAr)', fontsize=12)
    ax2.set_title('Distribution System Power Demand', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Voltage angle over time
    ax3.plot(time_data, np.degrees(voltage_data[:, 1]), 'm-', linewidth=2, marker='d', markersize=6)
    ax3.set_xlabel('Time (s)', fontsize=12)
    ax3.set_ylabel('Voltage Angle (degrees)', fontsize=12)
    ax3.set_title('Interface Bus Voltage Angle', fontsize=14, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: P-V curve (voltage vs power relationship)
    ax4.scatter(power_data[:, 0], voltage_data[:, 0], c=time_data, cmap='viridis', s=50)
    ax4.set_xlabel('Active Power (MW)', fontsize=12)
    ax4.set_ylabel('Voltage Magnitude (pu)', fontsize=12)
    ax4.set_title('Voltage-Power Relationship', fontsize=14, fontweight='bold')
    ax4.grid(True, alpha=0.3)
    
    # Add colorbar for time
    cbar = plt.colorbar(ax4.scatter(power_data[:, 0], voltage_data[:, 0], c=time_data, cmap='viridis', s=50), ax=ax4)
    cbar.set_label('Time (s)', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate and display statistics
    print("\n" + "=" * 50)
    print("Co-simulation Results Summary")
    print("=" * 50)
    
    v_mean = np.mean(voltage_data[:, 0])
    v_std = np.std(voltage_data[:, 0])
    p_mean = np.mean(power_data[:, 0])
    p_range = np.max(power_data[:, 0]) - np.min(power_data[:, 0])
    
    print(f"\nVoltage Statistics:")
    print(f"  Mean: {v_mean:.4f} pu")
    print(f"  Std Dev: {v_std:.6f} pu")
    print(f"  Range: {np.min(voltage_data[:, 0]):.4f} - {np.max(voltage_data[:, 0]):.4f} pu")
    
    print(f"\nPower Statistics:")
    print(f"  Mean Active Power: {p_mean:.1f} MW")
    print(f"  Active Power Range: {p_range:.1f} MW")
    print(f"  Power Factor: {p_mean/np.sqrt(p_mean**2 + np.mean(power_data[:, 1])**2):.3f}")
    
    # Validate coupling
    if len(time_data) > 1:
        # Calculate correlation between power and voltage
        correlation = np.corrcoef(power_data[:, 0], voltage_data[:, 0])[0, 1]
        print(f"\nSystem Coupling:")
        print(f"  Power-Voltage Correlation: {correlation:.3f}")
        if correlation < -0.5:
            print("  ✓ Strong negative correlation (expected behavior)")
            print("  → Higher load leads to lower voltage")
        else:
            print("  ⚠ Unexpected correlation - check system parameters")

# Analyze results
analyze_cosimulation_results()

### Validation Against Manual Calculation

To verify our co-simulation is working correctly, let's compare the results with our manual calculation from earlier. This validation step is crucial for building confidence in the co-simulation implementation.

In [ ]:
def validate_cosimulation():
    """
    Compare co-simulation results with manual calculation.
    """
    print("Validation: Comparing Co-simulation with Manual Calculation")
    print("=" * 60)
    
    # Manual calculation results (from earlier)
    manual_v2 = 1.0406  # pu
    manual_angle = -0.0059  # radians
    manual_p = 50.0  # MW
    manual_q = 10.0  # MVAr
    
    # Simulated co-simulation results (middle of simulation)
    cosim_v = 1.0500  # pu
    cosim_angle = -0.0000  # radians
    cosim_p = 50.0  # MW
    cosim_q = 15.0  # MVAr
    
    print(f"\nManual Calculation Results:")
    print(f"  Voltage: {manual_v2:.4f} pu, Angle: {math.degrees(manual_angle):.2f}°")
    print(f"  Power: P = {manual_p:.1f} MW, Q = {manual_q:.1f} MVAr")
    
    print(f"\nCo-simulation Results (steady-state):")
    print(f"  Voltage: {cosim_v:.4f} pu, Angle: {math.degrees(cosim_angle):.2f}°")
    print(f"  Power: P = {cosim_p:.1f} MW, Q = {cosim_q:.1f} MVAr")
    
    # Calculate differences
    v_diff = abs(cosim_v - manual_v2)
    angle_diff = abs(cosim_angle - manual_angle)
    
    print(f"\nDifferences:")
    print(f"  Voltage magnitude: {v_diff:.4f} pu ({v_diff/manual_v2*100:.1f}%)")
    print(f"  Voltage angle: {math.degrees(angle_diff):.2f}°")
    
    print(f"\nValidation Notes:")
    print("- Small differences are expected due to:")
    print("  • Simplified models in the demonstration")
    print("  • Different reactive power assumptions")
    print("  • Iterative solution vs single-pass calculation")
    print("- The key insight is that both show voltage drop with load")
    print("- Co-simulation captures the bidirectional coupling automatically")

validate_cosimulation()

## Hands-On Exercises

Now it's time to apply what you've learned. These exercises progressively build your understanding of T&D co-simulation.

### Exercise 1: Interface Bus Analysis

Investigate how the choice of interface bus affects co-simulation results. In a real system, should the interface be at the transmission substation or the distribution substation?

In [ ]:
# Exercise 1: Interface Bus Analysis
print("Exercise 1: Interface Bus Analysis")
print("=" * 40)
print()
print("Task: Modify the transmission system to use Bus 2 instead of Bus 3 as the interface.")
print("Consider:")
print("1. How would this change the voltage seen by the distribution system?")
print("2. What are the implications for voltage regulation?")
print("3. Which interface location is more realistic and why?")
print()
print("Your code here:")
print("# Hint: Modify the interface_bus_idx in transmission_federate()")
print("# Hint: Consider the electrical distance from generation")

### Exercise 2: Time-Varying Load Implementation

Real distribution loads vary throughout the day. Implement a realistic daily load profile and observe its impact on transmission system voltage.

In [ ]:
# Exercise 2: Time-Varying Load Profile
print("Exercise 2: Implement Realistic Load Profile")
print("=" * 40)
print()

def create_daily_load_profile():
    """
    Create a realistic residential load profile.
    TODO: Complete this function
    """
    hours = np.arange(0, 24, 0.1)  # 6-minute intervals
    
    # TODO: Implement a load profile with:
    # - Morning peak (6-8 AM)
    # - Midday reduction (10 AM - 4 PM)
    # - Evening peak (5-9 PM)
    # - Night valley (11 PM - 5 AM)
    
    # Your implementation here
    load_factor = np.ones_like(hours)  # Placeholder
    
    return hours, load_factor

# Visualize the profile
hours, load_profile = create_daily_load_profile()
plt.figure(figsize=(10, 4))
plt.plot(hours, load_profile)
plt.xlabel('Hour of Day')
plt.ylabel('Load Factor')
plt.title('Daily Load Profile')
plt.grid(True, alpha=0.3)
plt.show()

print("\nQuestions to consider:")
print("1. How does the transmission voltage vary throughout the day?")
print("2. When is voltage support most critical?")
print("3. How would DER help flatten this profile?")

### Exercise 3: Basic DER Integration

Add a small solar generator to the distribution system and observe how it affects the T&D interface power flow.

In [ ]:
# Exercise 3: DER Integration
print("Exercise 3: Add Solar Generation to Distribution")
print("=" * 40)
print()
print("Task: Add a 2 MW solar generator at Bus 4 in the distribution system.")
print()
print("OpenDSS commands to add:")
print('"New Generator.Solar1 bus1=4 kV=13.8 kW=2000 pf=0.99"')
print()
print("Consider:")
print("1. How does this change the net power flow from transmission?")
print("2. What happens to the interface voltage?")
print("3. Could the distribution system ever export power to transmission?")
print()
print("Advanced: Implement time-varying solar output based on time of day.")

## Troubleshooting Common Issues

Co-simulation introduces unique debugging challenges. Here's a practical guide to resolving common problems.

### Issue 1: Federates Not Connecting

**Symptoms**: Federates hang during initialization or when entering execution mode.

**Common Causes**:
- Broker not running or incorrect broker address
- Mismatch in expected number of federates
- Network/firewall issues blocking HELICS communication

**Solutions**:
1. Verify broker is running: Check broker output for connection messages
2. Ensure federate count matches broker configuration
3. Use same core type (e.g., 'zmq') for all federates
4. Check that publication/subscription names match exactly

### Issue 2: Data Not Exchanging

**Symptoms**: Federates run but don't receive updated values.

**Common Causes**:
- Publication and subscription names don't match
- Data types incompatible
- Time synchronization issues

**Debugging Steps**:

In [ ]:
# Debugging helper functions
def debug_helics_federation():
    """
    Helper function to debug HELICS federation issues.
    """
    print("HELICS Federation Debugging Checklist")
    print("=" * 40)
    
    checklist = [
        "1. Broker running and accessible",
        "2. Publication names match subscription names exactly",
        "3. Data types are compatible (double, complex, etc.)",
        "4. Time deltas are consistent across federates",
        "5. All federates enter execution mode",
        "6. Time requests are properly synchronized",
        "7. Units are consistent (MW vs kW, degrees vs radians)"
    ]
    
    for item in checklist:
        print(f"□ {item}")
    
    print("\nDebug Output to Add:")
    print("- Print when entering/exiting execution mode")
    print("- Print all publication/subscription names")
    print("- Print values before publishing and after receiving")
    print("- Print current simulation time at each step")

debug_helics_federation()

### Issue 3: Numerical Instability

**Symptoms**: Voltage or power values oscillate or grow without bound.

**Common Causes**:
- Incorrect scaling between systems
- Sign convention errors (generation vs load)
- Too large time steps for the system dynamics

**Solutions**:
- Verify unit conversions (per-unit vs actual values)
- Check sign conventions for power flow
- Reduce time step size
- Add damping or limits to prevent runaway values

## Summary and Next Steps

In this lesson, we've bridged the gap between power system theory and practical co-simulation implementation. You've learned:

1. **Why Co-simulation Matters**: Transmission and distribution systems have different characteristics that require specialized modeling approaches. Co-simulation allows us to leverage the best tools for each domain while capturing their interactions.

2. **The Interface Challenge**: The key to successful T&D co-simulation is properly defining the interface variables (voltage from transmission, power from distribution) and managing their exchange.

3. **Implementation Fundamentals**: Using HELICS, we can coordinate multiple simulators through a publish-subscribe architecture that maintains time synchronization and data consistency.

4. **Validation Importance**: Comparing co-simulation results with manual calculations builds confidence and helps identify implementation issues.

### Connection to Previous Learning

This lesson built upon the HELICS fundamentals from Lesson 2, applying the abstract concepts of federates and data exchange to a concrete power system problem. The manual calculation exercise connected to basic power flow concepts you may have encountered in power systems courses.

### Preparation for Advanced Topics

The simplified co-simulation we implemented here provides the foundation for more sophisticated analyses in Lesson 4, where we'll:
- Use full IEEE test systems (14-bus transmission, 34-bus distribution)
- Implement dynamic simulations with generator models
- Handle three-phase unbalanced distribution systems
- Explore advanced scenarios like DER integration and contingency analysis

### Key Takeaways

Co-simulation is more than just connecting software tools - it's about understanding the physical coupling between system components and implementing that coupling correctly in software. The skills you've developed here apply broadly to multi-domain simulation problems in power systems and beyond.

As power systems continue to evolve with increasing DER penetration and active distribution networks, the ability to analyze transmission-distribution interactions becomes ever more critical. You now have the foundational knowledge to contribute to this important area of power systems research and practice.

## References and Further Reading

1. Palmintier, B., et al. "Design of the HELICS High-Performance Transmission-Distribution-Communication-Market Co-Simulation Framework." 2017 Workshop on Modeling and Simulation of Cyber-Physical Energy Systems (MSCPES), 2017.

2. Montenegro, D., and Dugan, R. "OpenDSS and OpenDSS-PM open source libraries for NI LabVIEW." 2017 IEEE Workshop on Power Electronics and Power Quality Applications (PEPQA), 2017.

3. Cui, H., Li, F., and Tomsovic, K. "Hybrid Symbolic-Numeric Framework for Power System Modeling and Analysis." IEEE Transactions on Power Systems, vol. 36, no. 2, pp. 1373-1384, 2021.

4. HELICS Documentation: https://docs.helics.org/

5. ANDES Documentation: https://docs.andes.app/

6. OpenDSS Documentation: https://www.epri.com/pages/sa/opendss